# Whitebox testing of robustness against BoundaryAttack


### Importing required packages

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import tensorflow as tf


from art.attacks import BoundaryAttack
from art.classifiers import KerasClassifier, BlackBoxClassifier
from art.utils import load_mnist

tf.compat.v1.disable_eager_execution()

Using TensorFlow backend.


### Load Dataset

In [2]:
# Step 1: Load the MNIST dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

### Creating a sample model
A sample model is used to demonstrate the use of ART library
Model to be tested will be replaced here

In [3]:
# Step 2: Create the model

model = Sequential()

model.add(Conv2D(4,5,padding='same',input_shape = [28, 28, 1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(10,5, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### Wrapping of ART classifier
Wrapping is necessary to use ART functions

In [4]:
# Step 3: wrap ART tf classifier
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)

Epoch 1/10
469/468 [==============================] - 6s 12ms/step - loss: 0.3632 - accuracy: 0.8931
Epoch 2/10
469/468 [==============================] - 5s 10ms/step - loss: 0.1159 - accuracy: 0.9648
Epoch 3/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0754 - accuracy: 0.9770
Epoch 4/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0629 - accuracy: 0.9799
Epoch 5/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0508 - accuracy: 0.9845
Epoch 6/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0405 - accuracy: 0.9871
Epoch 7/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0355 - accuracy: 0.9889
Epoch 8/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0325 - accuracy: 0.9895
Epoch 9/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0281 - accuracy: 0.9909
Epoch 10/10
469/468 [==============================] - 5s 10ms/step - loss: 0.0256 - accura

### Evaluating model based on original Dataset

In [5]:
# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

Accuracy on benign test examples: 98.7%


### Creating Adversarial samples
Using Boundary Attack

In [ ]:
# Step 6: Generate adversarial test examples
attack = BoundaryAttack(classifier)
x_test_adv = attack.generate(x=x_test, y=y_test)

### Evaluating model on adversarial samples

In [24]:
# Step 7: Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

Accuracy on adversarial test examples: 2.75%


### Evaluation results
- Using accuracy as an evaluation metric, we can see that the sample model's performance dropped by a huge amount after going through adversarial samples.
- Results is normal as sample model does not have defence against adversarial attacks


## Adding defence layers based off advesarial attack
we will now add basing adversarial training defence to the model and evaluate its robustness using accuracy

### Creating more data for adversarial training
as adversarial datasets from x_train will be used to evaluate the trained model, we will need to create more data for adversarial training of model.
Thus, we will use Keras image augmentation to create more data for adversarial training.

In [25]:
#importing packages
from art.defences import AdversarialTrainer
from keras.preprocessing.image import ImageDataGenerator
from art.data_generators import KerasDataGenerator

In [26]:
# Build a Keras image augmentation object and wrap it in ART
batch_size = 50
datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.125, height_shift_range=0.125,
                             fill_mode='constant', cval=0.)
datagen.fit(x_train)
art_datagen = KerasDataGenerator(datagen.flow(x=x_train, y=y_train, batch_size=batch_size, shuffle=True),
                                 size=x_train.shape[0], batch_size=batch_size)


### Creating and training adversarial trainer

In [ ]:
# Create adversarial trainer and perform adversarial training
adv_trainer = AdversarialTrainer(classifier, attacks=attack, ratio=1.)
adv_trainer.fit_generator(art_datagen, nb_epochs=2)

Epoch 1/1
1/1 [==============================] - 0s 8ms/step - loss: 14.7895 - accuracy: 0.2000
Epoch 1/1
1/1 [==============================] - 0s 11ms/step - loss: 14.3750 - accuracy: 0.1200
Epoch 1/1
1/1 [==============================] - 0s 19ms/step - loss: 6.8621 - accuracy: 0.1600
Epoch 1/1
1/1 [==============================] - 0s 18ms/step - loss: 5.2989 - accuracy: 0.2800
Epoch 1/1
1/1 [==============================] - 0s 18ms/step - loss: 4.0093 - accuracy: 0.2600
Epoch 1/1
1/1 [==============================] - 0s 18ms/step - loss: 4.3352 - accuracy: 0.1600
Epoch 1/1
1/1 [==============================] - 0s 9ms/step - loss: 2.5264 - accuracy: 0.1600
Epoch 1/1
1/1 [==============================] - 0s 12ms/step - loss: 1.6886 - accuracy: 0.2600
Epoch 1/1
1/1 [==============================] - 0s 10ms/step - loss: 1.9286 - accuracy: 0.3200
Epoch 1/1
1/1 [==============================] - 0s 12ms/step - loss: 1.5710 - accuracy: 0.4200
Epoch 1/1
1/1 [=========================

In [ ]:
# Step 7: Evaluate the ART classifier on adversarial test examples

predictions_adv = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions_adv, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

### Evaluation results
- As we can see, the model performed better in terms of accuracy when it is adversarially trained against adversarial attacks
